In [2]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

import os
import logging
from knockpy.knockoff_filter import KnockoffFilter
import pandas as pd
import numpy as np
import warnings
import knockpy
import ray

import pandas as pd

from ml.models import common
from utils import helper, config, rayer, kaggle_dataset_helper
from ml.xai.non_model.knockoff_filter.config import knockoffsettings
from ml.xai.non_model.knockoff_filter.knockoff_sim_ray import sim_knockoffs



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# ray.init(address="ray://10.10.35.92:10001")

In [ ]:
rayer.get_global_cluster(num_cpus=4)

Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2022-11-30 07:53:24,521	WARNING dataclient.py:395 -- Encountered connection issues in the data channel. Attempting to reconnect.
2022-11-30 07:53:34,248	ERROR dataclient.py:322 -- Unrecoverable error in data channel.


In [5]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
#ss = StandardScaler()
#X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
#X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)

# X = X.to_numpy()
# y = y.values.ravel()


In [7]:
ds_train = pd.read_csv('/home/wasif/python-asd/xai/train.csv', sep = ',')
ds_train = common.label_encode(ds_train)
ds_train = ds_train.fillna(-1)


df_X = ds_train.loc[:, ~ds_train.columns.isin(['target', 'ID_code'])]
df_y = ds_train['target']



np.random.seed(knockoffsettings.SEED)
Sigma = None

#X = df[df.columns[df.columns!='y']]
#y = df[df.columns[df.columns=='y']]
X = df_X
y = df_y
# X = X.to_numpy()
# y = y.values.ravel()


# ds_train.head()

In [6]:
param = {}
param['fdr'] = knockoffsettings.FDR
# param
# simmulate(param, 0, X, y)

In [9]:
df = sim_knockoffs(param, 100, X, y)

(__worker__ pid=32483) itr: 2, ksampler:gaussian, fstat:lasso
(__worker__ pid=32481) itr: 1, ksampler:gaussian, fstat:lasso
(__worker__ pid=32484) itr: 4, ksampler:gaussian, fstat:lasso
(__worker__ pid=32482) itr: 3, ksampler:gaussian, fstat:lasso
(__worker__ pid=32483) itr: 5, ksampler:gaussian, fstat:lasso
(__worker__ pid=32484) itr: 6, ksampler:gaussian, fstat:lasso
(__worker__ pid=32481) itr: 7, ksampler:gaussian, fstat:lasso
(__worker__ pid=32482) itr: 8, ksampler:gaussian, fstat:lasso
(__worker__ pid=32483) itr: 10, ksampler:gaussian, fstat:lasso
(__worker__ pid=32484) itr: 9, ksampler:gaussian, fstat:lasso
(__worker__ pid=32481) itr: 11, ksampler:gaussian, fstat:lasso
(__worker__ pid=32482) itr: 12, ksampler:gaussian, fstat:lasso
(__worker__ pid=32484) itr: 13, ksampler:gaussian, fstat:lasso
(__worker__ pid=32483) itr: 14, ksampler:gaussian, fstat:lasso
(__worker__ pid=32481) itr: 15, ksampler:gaussian, fstat:lasso
(__worker__ pid=32482) itr: 16, ksampler:gaussian, fstat:lasso
(

In [16]:
df.sort_values(['W'], ascending=False)

,rejections,W,attr
0,0.0,620711.473263,new_cases_per_million
30,1.0,174014.116558,Cardiovascular diseases
20,1.0,31324.059838,Alcohol use disorders
35,0.0,20173.867311,Acute hepatitis
12,1.0,14064.967651,Interpersonal violence
39,1.0,11994.707461,air_polution
36,0.0,7893.854539,Alzheimer's disease and other dementias
11,0.0,6956.630812,Drowning
29,0.0,6252.844541,Protein-energy malnutrition
34,1.0,6087.138993,Digestive diseases


In [59]:
df_results = pd.DataFrame()        

for result in results:
    df_results = pd.concat([df_results, pd.DataFrame(result)], axis=0)
    
df_results.columns = ['rej', 'w', 'itr']

In [60]:
df_results

,rej,w,itr
0,1.0,0.004988,1.0
1,0.0,0.000001,1.0
2,1.0,9.804645,1.0
3,0.0,0.000001,1.0
4,1.0,0.000080,1.0
...,...,...,...
35,0.0,-59238.064386,2.0
36,0.0,18057.707824,2.0
37,0.0,-1300.593139,2.0
38,0.0,60.379812,2.0


In [54]:
pd.concat([pd.DataFrame(results[0]), pd.DataFrame(results[1])])

,0,1,2
0,1.0,0.004988,1.0
1,0.0,0.000001,1.0
2,1.0,9.804645,1.0
3,0.0,0.000001,1.0
4,1.0,0.000080,1.0
...,...,...,...
35,0.0,-0.000002,2.0
36,0.0,-0.000003,2.0
37,0.0,-0.000001,2.0
38,0.0,-0.000004,2.0


In [ ]:
for base_model in self.base_models:
    lazy_results.append(__run_discoveries__.remote(base_model, score_func, threshold))


self.base_models = ray.get(lazy_results)


In [25]:

# for ksampler in knockoffsettings.KSAMPLER:
ksampler = knockoffsettings.KSAMPLER[0]
fstat = knockoffsettings.FSTATS[0]
fdr = knockoffsettings.FDR


In [26]:
kfilter = KnockoffFilter(
    ksampler = ksampler,
    fstat = fstat,
)

rejections = kfilter.forward(
    X = X,
    y = y,
    Sigma = Sigma,
    fdr = fdr
)

In [27]:
rejections

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [ ]:


# print( 'itr: ' + str(index) + ', ksampler:' + str(ksampler) + ', fstat:' + str(fstat) )

if ksampler != 'metro':
    kfilter = KnockoffFilter(
        ksampler = ksampler,
        fstat = fstat,
    )

    rejections = kfilter.forward(
        X = X,
        y = y,
        Sigma = Sigma,
        fdr = fdr
    )